In [ ]:
# RHF Solver

import numpy as np
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock3.block2.io import MPSTools, MPOTools

L = 6
N = 6
TWOS = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOS)

t = 1
U = 2

b = driver.expr_builder()

# hopping term
b.add_term("(C+D)0",
    np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten(),
    [np.sqrt(2) * -t] * 2 * (L - 1))

# onsite term
b.add_term("((C+(C+D)0)1+D)0",
    np.array([[i, ] * 4 for i in range(L)]).flatten(),
    [U] * L)

mpo = driver.get_mpo(b.finalize(), iprint=2)
ket = driver.get_random_mps(tag="KET", bond_dim=400, nroots=1)

def run_dmrg(driver, mpo):
    bond_dims = [200] * 4 + [400] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(
        mpo,
        ket,
        n_sweeps=25,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=2,
    )

energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)



In [1]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

print(gto.energy_nuc(mol))
print(ecore)


23.572439395527272
23.572439395527272


In [1]:
# block2 itg for DMET fragments and baths

def get_dmet_integer_rhf(frag_coeff, bath_coeff, core_coeff, mf):

    mo_coeff = np.hstack((core_coeff, frag_coeff, bath_coeff))
    ncore = core_coeff.shape[1]
    
    class PseudoMF(mf.__class__):
        def __init__(self, mol):
            super(PseudoMF, self).__init__(mol)
            self.mo_coeff = mo_coeff

    mf = PseudoMF()
    ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf, ncore)
    return h1e, g2e


In [16]:
#Block2
import numpy as np
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock3.block2.io import MPSTools, MPOTools
from pyblock3.algebra.core import SparseTensor

L = 6
N = 6
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

t = 1
U = 6

b = driver.expr_builder()

# hopping term
b.add_term("cd",
    np.array([[[i, (i+1)%L], [(i+1)%L, i]] for i in range(L)]).flatten(),
    [-t] * 2 * (L))
b.add_term("CD",
    np.array([[[i, (i+1)%L], [(i+1)%L, i]] for i in range(L)]).flatten(),
    [-t] * 2 * (L))

# onsite term
b.add_term("cdCD",
    np.array([[i, ] * 4 for i in range(L)]).flatten(),
    [U] * L)

mpo = driver.get_mpo(b.finalize(), iprint=2, add_ident=False)
ket = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)

def run_dmrg(driver, mpo):
    bond_dims = [250] * 4 + [500] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(
        mpo,
        ket,
        n_sweeps=25,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=2,
    )

energies = run_dmrg(driver, mpo)
print(energies)

dm1a, dm1b = driver.get_1pdm(ket)
dm2aa, dm2ab, dm2bb = driver.get_2pdm(ket)
dm2aa = dm2aa.transpose(0,3,1,2)
dm2ab = dm2ab.transpose(0,3,1,2)
dm2bb = dm2bb.transpose(0,3,1,2)


Build MPO | Nsites =     6 | Nterms =         30 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     6 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.001 T = 0.017
 Site =     1 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.7500 Tmvc = 0.000 T = 0.004
 Site =     2 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8500 Tmvc = 0.000 T = 0.001
 Site =     3 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8500 Tmvc = 0.000 T = 0.002
 Site =     4 /     6 .. Mmpo =     6 DW = 0.00e+00 NNZ =       15 SPT = 0.7500 Tmvc = 0.000 T = 0.002
 Site =     5 /     6 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.003
Ttotal =      0.029 Tmvc-total = 0.001 MPO bond dimension =    10 MaxDW = 0.00e+00
NNZ =           72 SIZE =          332 SPT = 0.7831

Rank =     0 Ttotal =      0.053 MPO method = FastBipartite bond dimension =      10 NNZ =           72 SIZE =          332 SPT = 0.7831
Environment initi

In [17]:
import sys
sys.path.append("..")

from dmet.model import hubbard
from pyscf.fci import fci_slow, direct_spin1

t = 1
L = 6
U = 6

mol = hubbard.HubbardMole(lx=L, ly=1, t=t, U=U)
mol.nelectron = L
mf = hubbard.RHF_SpinHubbard(mol)
mf.pbc = True
mf.kernel()
h1e = mf.get_hcore()
eri = mf._eri
e, hop, diag, c = fci_slow.kernel(h1e, eri, L, L)
dm_fci, dm2_fci = direct_spin1.make_rdm12(c, L, (L//2,L//2))


converged SCF energy = 0.999999999999998


Initialize <dmet.model.hubbard.HubbardMole object at 0x7fbca8115af0> in <dmet.model.hubbard.RHF_SpinHubbard object at 0x7fbca81159d0>


In [18]:
np.linalg.norm(dm2aa + dm2ab * 2 + dm2bb -dm2_fci)

7.100452595215451e-06

In [3]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2.algebra.io import MPSTools, MPOTools

from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

print(h1e.shape)
print(g2e.shape)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e)


(10, 10)
(10, 10, 10, 10)
integral symmetrize error =  0.0
integral cutoff error =  1.9890268009433636e-20
mpo terms =       3750

Build MPO | Nsites =    10 | Nterms =       3750 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.001 T = 0.318
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      175 SPT = 0.9116 Tmvc = 0.003 T = 0.171
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      291 SPT = 0.9599 Tmvc = 0.002 T = 0.137
 Site =     3 /    10 .. Mmpo =   162 DW = 0.00e+00 NNZ =      675 SPT = 0.9621 Tmvc = 0.002 T = 0.164
 Site =     4 /    10 .. Mmpo =   222 DW = 0.00e+00 NNZ =     1199 SPT = 0.9667 Tmvc = 0.003 T = 0.045
 Site =     5 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =      979 SPT = 0.9763 Tmvc = 0.002 T = 0.045
 Site =     6 /    10 .. Mmpo =   106 DW = 0.00e+00 NNZ =      503 SPT = 0.9745 Tmvc = 0.001 T = 0.022
 Site =     7 /    10 .. Mmpo =    58 DW = 0.00e+00 NNZ =